In [3]:
import numpy as np 
import math as m 
import timeit 

def CRR_Tree(S,v,n,T): 
    h = T/n    # time step
    u = np.exp( v * np.sqrt(h))            # u = e ^ (v * sqrt(h)), v = volatility (sigma )
    d = 1/u   
    Tree = np.zeros((n+1, n+1))            # setup: n steps trees have n + 1 nodes 
    
    for j in range(n+1):   # 分成幾個 step : columns
        for i in range(j+1):  # 有幾個j就有幾個i  # i: # of down, j: # of total moves , j-i : # of up
            Tree[i,j] = S * m.pow(d,i) * m.pow(u,j-i) 
    
    print('n = ' + str(n) + ':\n',np.matrix(np.round(Tree,2)))         
    return Tree

In [4]:
S = 40
v = 0.3
n = 2
T = 1 
x = CRR_Tree(S,v,n,T)

n = 2:
 [[40.   49.45 61.14]
 [ 0.   32.35 40.  ]
 [ 0.    0.   26.17]]


### Check 

In [12]:
h = T/n    # time step
u = np.exp( v * np.sqrt(h))    # u = e ^ (v * sqrt(h))
d = 1/u   

Tree = np.zeros((n+1, n+1))   # setup: n steps trees have n + 1 nodes 
Tree

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

In [11]:
# # i: # of down, j: # of total moves , j-i : # of up 
# import time 

# for j in range(n+1): 
#         for i in range(j+1): 
#             Tree[i,j] = S * m.pow(d,i) * m.pow(u,j-i) 
#                             # d**i * u**j-i
#             print('n = ' + str(n) + ':\n',np.matrix(np.round(Tree,2)))    
#             time.sleep(1)     


### Pricing European Call 

In [8]:
import numpy as np
import math as m

def BinoEuroCall(S, K, v, r, n, T):
    # Calculate time step
    h = T / n

    # Calculate up and down factors
    u = m.exp(v * np.sqrt(h))
    d = 1 / u    # CRR assumption 

    # Calculate risk-neutral probability ( nomoral q )
    q = (np.exp(r * h) - d) / (u - d)                  # q = R-d / (u-d)  or q = (e ^ (rh) - d) / u - d 

    
    # CRR 
    Tree = np.zeros((n + 1, n + 1))    
    for j in range(n + 1):
        for i in range(j + 1):
            Tree[i, j] = S * m.pow(d, i) * m.pow(u, j - i)

    
    
    # Call Price Tree 
    Call = np.zeros((n + 1, n + 1))
    
    for j in range(n + 1, 0, -1):    # backward induction : from n+1 to 0
        for i in range(j):  
            
            if j == n + 1:   # when j = n+1, we're at T. calculate terminal payoff  # a specific node 
                # At maturity, payoff is max(Tree - K, 0)
                Call[i, j - 1] = max(Tree[i, j - 1] - K, 0)   
            else:
                # Discounted expected value for intermediate nodes
                Call[i, j - 1] = np.exp(-r * h) * (q * Call[i, j] + (1 - q) * Call[i + 1, j])  # i : num of downs 

    # Print stock price tree and option value tree
    print('Stock Price Tree (n = ' + str(n) + '):\n', np.matrix(np.round(Tree, 2)))
    print('-------------------------------------------')
    print('Option Value Tree (n = ' + str(n) + '):\n', np.matrix(np.round(Call, 2)))

    
    # result 
    return Call[0, 0]


In [9]:
S = 40
K = 40 
r = 0.08 
v = 0.3 
T = 1 
n = 2 
cp = BinoEuroCall(S,K,v,r,n,T) 
print('The binomial European call price is ' + str(round(cp,6)))   # change the rounding to see more decimal 

Stock Price Tree (n = 2):
 [[40.   49.45 61.14]
 [ 0.   32.35 40.  ]
 [ 0.    0.   26.17]]
-------------------------------------------
Option Value Tree (n = 2):
 [[ 5.75 11.02 21.14]
 [ 0.    0.    0.  ]
 [ 0.    0.    0.  ]]
The binomial European call price is 5.745862


In [10]:
# # just for visualization


# h = T / n
# u = np.exp(v * np.sqrt(h))
# d = 1 / u
# q = (np.exp(r * h) - d) / (u - d)   

# Tree = np.zeros((n + 1, n + 1))     
# for j in range(n + 1):
#     for i in range(j + 1):
#         Tree[i, j] = S * m.pow(d, i) * m.pow(u, j - i)

# Call = np.zeros((n + 1, n + 1))

# for j in range(n + 1, 0, -1):    
#         for i in range(j):
            
#             if j == n + 1:   
#                 Call[i, j - 1] = max(Tree[i, j - 1] - K, 0)  
            
#             else:
#                 Call[i, j - 1] = m.exp(-r * h) * (q * Call[i, j] + (1 - q) * Call[i + 1, j])  # i: number of downs 
            
#             print('n = ' + str(n) + '):\n', np.matrix(np.round(Call, 2))) 
#             time.sleep(1)
            

### European Put 

In [38]:
import numpy as np
import math as m

def BinoEuroPut(S, K, v, r, n, T):
    ### Underlying asset tree 
    h = T / n

    u = np.exp(v * np.sqrt(h))
    d = 1 / u
    q = (m.exp(r * h) - d) / (u - d)   

    Tree = np.zeros((n + 1, n + 1))     ## CRR tree reamins the same 
    
    for j in range(n + 1):
        for i in range(j + 1):
            Tree[i, j] = S * m.pow(d, i) * m.pow(u, j - i)

    
    
    ### Put Tree 
    Put = np.zeros((n + 1, n + 1))
    
    for j in range(n + 1, 0, -1):    
        for i in range(j):
            
            if j == n + 1:   
                Put[i, j - 1] = max(K - Tree[i, j - 1], 0)   # Put : max(K - St, 0 ) 
            else:
                Put[i, j - 1] = m.exp(-r * h) * (q * Put[i, j] + (1 - q) * Put[i + 1, j])  # i : number of downs 

    # Print stock price tree and option value tree
    print('Stock Price Tree (n = ' + str(n) + '):\n', np.matrix(np.round(Tree, 2)))
    print('-------------------------------------------')
    print('Option Value Tree (n = ' + str(n) + '):\n', np.matrix(np.round(Put, 2)))

    
    # result 
    return Put[0, 0]


In [40]:
S = 97
K = 99
r = 0.03 
v = 0.3
T = 0.5
n = 10

result = BinoEuroPut(S,K,v,r,n,T) 
print(result)

Stock Price Tree (n = 10):
 [[ 97.   103.73 110.93 118.62 126.85 135.66 145.07 155.13 165.9  177.41
  189.72]
 [  0.    90.71  97.   103.73 110.93 118.62 126.85 135.66 145.07 155.13
  165.9 ]
 [  0.     0.    84.82  90.71  97.   103.73 110.93 118.62 126.85 135.66
  145.07]
 [  0.     0.     0.    79.32  84.82  90.71  97.   103.73 110.93 118.62
  126.85]
 [  0.     0.     0.     0.    74.17  79.32  84.82  90.71  97.   103.73
  110.93]
 [  0.     0.     0.     0.     0.    69.36  74.17  79.32  84.82  90.71
   97.  ]
 [  0.     0.     0.     0.     0.     0.    64.86  69.36  74.17  79.32
   84.82]
 [  0.     0.     0.     0.     0.     0.     0.    60.65  64.86  69.36
   74.17]
 [  0.     0.     0.     0.     0.     0.     0.     0.    56.72  60.65
   64.86]
 [  0.     0.     0.     0.     0.     0.     0.     0.     0.    53.04
   56.72]
 [  0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
   49.6 ]]
-------------------------------------------
Option Value Tree (n = 10):

In [15]:
S = 40
K = 41
r = 0.08 
v = 0.3 
T = 1 
n = 2 
CP = BinoEuroCall(S,K,v,r,n,T)   # call price 
print('=======================================================')

PP = BinoEuroPut(S,K,v,r,n,T) 

print('The binomial European put price is ' + str(round(PP,6)))   
print('The binomial European call price is ' + str(round(CP,6)))   

Stock Price Tree (n = 2):
 [[40.   49.45 61.14]
 [ 0.   32.35 40.  ]
 [ 0.    0.   26.17]]
-------------------------------------------
Option Value Tree (n = 2):
 [[ 5.47 10.5  20.14]
 [ 0.    0.    0.  ]
 [ 0.    0.    0.  ]]
Stock Price Tree (n = 2):
 [[40.   49.45 61.14]
 [ 0.   32.35 40.  ]
 [ 0.    0.   26.17]]
-------------------------------------------
Option Value Tree (n = 2):
 [[ 3.32  0.44  0.  ]
 [ 0.    7.04  1.  ]
 [ 0.    0.   14.83]]
The binomial European put price is 3.321813
The binomial European call price is 5.474043


In [34]:
### put call parity: C - P ( long call, short put ) = S - PV( k )
""" 
C : 
"""

In [22]:

CP - PP 

2.7395522883652044

In [23]:
S - K * np.exp(-r * T)

2.739552288365161

### Pricing American Put

In [21]:
import numpy as np
import math as m

def BinoAmerPut(S, K, v, r, n, T, div):  # div : continuous dividend yield --> necessary for american call 
    # Time step size
    h = T / n

    u = m.exp(v * m.sqrt(h))
    d = 1 / u
    q = (m.exp((r - div) * h) - d) / (u - d)  #  e^(r - delta) h / u - d 

    Tree = np.zeros((n + 1, n + 1))  # setup 
    
    for j in range(n + 1):
        for i in range(j + 1):
            Tree[i, j] = S * m.pow(d, i) * m.pow(u, j - i)

    # Put Tree 
    Put = np.zeros((n + 1, n + 1))
    Exer = np.zeros((n + 1, n + 1))  # check for early exercise 
    
    for j in range(n + 1, 0, -1):
        for i in range(j):
            
            if j == n + 1:
                # Option price at maturity 
                Put[i, j - 1] = max(K - Tree[i, j - 1], 0)  # put: K-St
            
            else:
                Put[i, j - 1] = max(K - Tree[i, j - 1],  # early exercise value of american put 
                                    np.exp(-r * h) * (q * Put[i, j] + (1 - q) * Put[i + 1, j]))  # the waiting value: based on European put 


                # for visualizing early exercise 
                if K - Tree[i, j - 1] > m.exp(-r * h) * (q * Put[i, j] + (1 - q) * Put[i + 1, j]): 
                    Exer[i, j - 1] = 1 
                    
                    
    # Print stock price tree
    print('Stock Price Tree (n = ' + str(n) + '):\n', np.matrix(np.round(Tree, 2)))
    
    print('-------------------------------------------')

    # Print option price tree for American put
    print('Put Option Price Tree (n = ' + str(n) + '):\n', np.matrix(np.round(Put, 2)))
    
    print('-------------------------------------------')

    # early exer points 
    print('n = ' + str(n) + '):\n', np.matrix(Exer))   
    
    

    # Return the option price at the root node
    return Put[0, 0]


In [22]:
# Parameters
S = 100          # Initial stock price
K = 101         # Strike price
v = 0.2         # Volatility
r = 0.05         # Risk-free interest rate
T = 2            # Time to maturity
n = 2           # Number of steps in the binomial tree
div = 0          # Dividend yield

# Calculate the binomial American put option price
App = BinoAmerPut(S, K, v, r, n, T, div)

# Print the result
print(f'The binomial American put price is {round(App, 3)}')


Stock Price Tree (n = 2):
 [[100.   122.14 149.18]
 [  0.    81.87 100.  ]
 [  0.     0.    67.03]]
-------------------------------------------
Put Option Price Tree (n = 2):
 [[ 7.91  0.4   0.  ]
 [ 0.   19.13  1.  ]
 [ 0.    0.   33.97]]
-------------------------------------------
n = 2):
 [[0. 0. 0.]
 [0. 1. 0.]
 [0. 0. 0.]]
The binomial American put price is 7.908


In [23]:
App   # result 

7.907903947307855

### American Call 

In [34]:
import numpy as np
import math as m

def BinoAmerCall(S, K, v, r, n, T, div):  # div : continuous dividend yield --> necessary for american call 
    # Time step size
    dt = T / n

    # Up and down factors
    u = m.exp(v * m.sqrt(dt))
    d = 1 / u
    q = (m.exp((r - div) * dt) - d) / (u - d)

    # Initialize stock price tree
    Tree = np.zeros((n + 1, n + 1))
    for j in range(n + 1):
        for i in range(j + 1):
            Tree[i, j] = S * m.pow(d, i) * m.pow(u, j - i)

    # Put Tree 
    Call = np.zeros((n + 1, n + 1))
    Exer = np.zeros((n + 1, n + 1)) 
    
    for j in range(n + 1, 0, -1):
        for i in range(j):
            
            if j == n + 1:
                Call[i, j - 1] = max(Tree[i, j - 1] - K , 0)  # Call : St- K 
            
            else:
                Call[i, j - 1] = max(Tree[i, j - 1] - K ,  # early exercise value of american call 
                                    m.exp(-r * dt) * (q * Call[i, j] + (1 - q) * Call[i + 1, j]))  # the waiting value: based on European call

            
                if Tree[i, j - 1] - K  > m.exp(-r * dt) * (q * Call[i, j] + (1 - q) * Call[i + 1, j]): 
                    Exer[i, j - 1] = 1 
                    
                    
    # Print stock price tree
    print('Stock Price Tree (n = ' + str(n) + '):\n', np.matrix(np.round(Tree, 2)))
    
    print('-------------------------------------------')

    # Print option price tree for American Call 
    print('Option Price Tree (n = ' + str(n) + '):\n', np.matrix(np.round(Call, 2)))
    
    print('-------------------------------------------')

    # print('n = ' + str(n) + '):\n', np.matrix(Exer))
    
    

    # Return the option price at the root node
    return Call[0, 0]


In [31]:
# Parameters
S = 100          # Initial stock price
K = 101         # Strike price
v = 0.2          # Volatility
r = 0.05        # Risk-free interest rate
T = 2            # Time to maturity
n = 2            # Number of steps in the binomial tree
div = 0          # Dividend yield

# Calculate the binomial American put option price
Acp = BinoAmerCall(S, K, v, r, n, T, div)

# Print the result
print(f'The binomial American call price is {round(Acp, 3)}')


Stock Price Tree (n = 2):
 [[100.   122.14 149.18]
 [  0.    81.87 100.  ]
 [  0.     0.    67.03]]
-------------------------------------------
Option Price Tree (n = 2):
 [[14.54 26.47 48.18]
 [ 0.    0.    0.  ]
 [ 0.    0.    0.  ]]
-------------------------------------------
The binomial American call price is 14.54


### Put Call parity : 
- Put call parity didn't apply to American Options 
- European options can only be exercised at maturity
The timing of cash flows is known with certainty
The relationship can be enforced through a risk-free arbitrage strategy


For American options, the relationship is different:


Early exercise is possible
The timing of cash flows becomes uncertain
Instead of equality, we get inequalities

In [35]:
S = 95
K = 96
v = 0.26
r = 0.05
T = 2 
n = 2 
div = 0.05

result = BinoAmerCall(S, K, v, r, n, T, div)
print(result)


Stock Price Tree (n = 2):
 [[ 95.   123.21 159.79]
 [  0.    73.25  95.  ]
 [  0.     0.    56.48]]
-------------------------------------------
Option Price Tree (n = 2):
 [[11.27 27.21 63.79]
 [ 0.    0.    0.  ]
 [ 0.    0.    0.  ]]
-------------------------------------------
11.267818334597871
